In [1]:
# Dependencies
import pymongo
import requests
import json
import pandas as pd

from config import api_key

from yelpapi import YelpAPI
yelp_api = YelpAPI(api_key)

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)



In [2]:
#term = 'Mexican restaurant bar'
#location = 'Astoria, NY'
term = 'restaurant'
#location = 'Huntington Beach, CA'
search_limit = 50
latitude = 33.669445
longitude = -117.823059
radius = 10000
response = yelp_api.search_query(term = term,
                                 limit = search_limit,
                                 latitude = latitude,
                                 longitude = longitude,
                                 radius = radius
                                )
                                


In [3]:
response

{'businesses': [{'id': 'NVB2X8t5Rie8S5AnzOnWJg',
   'alias': 'north-italia-irvine',
   'name': 'North Italia',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/hWa52yLwUAtiZh-xPYXGxg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/north-italia-irvine?adjust_creative=uWoa2sVIXxFlCSqOB-sCdg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=uWoa2sVIXxFlCSqOB-sCdg',
   'review_count': 3652,
   'categories': [{'alias': 'pizza', 'title': 'Pizza'},
    {'alias': 'italian', 'title': 'Italian'},
    {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 33.67173395544756,
    'longitude': -117.84513856994657},
   'transactions': ['delivery'],
   'price': '$$',
   'location': {'address1': '2957 Michelson Dr',
    'address2': '',
    'address3': '',
    'city': 'Irvine',
    'zip_code': '92612',
    'country': 'US',
    'state': 'CA',
    'display_address': ['2957 Michelson Dr', 'Irvine, CA 92612']},
  

In [4]:
response.keys()

dict_keys(['businesses', 'total', 'region'])

In [5]:
response['businesses'][0].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance'])

In [6]:
response['total']

2400

In [7]:
response['region']

{'center': {'longitude': -117.823059, 'latitude': 33.669445}}

In [8]:
# Define the 'restaurantes_db' database in Mongo
db = client.restaurantes_db

In [9]:
# Declare the collection
business = db.business

In [10]:
categories = []
transactions = []

for biz in response['businesses']:
    name = biz['name']
    address = biz['location']['address1']
    city = biz['location']['city']
    state = biz['location']['state']
    zip_code = biz['location']['zip_code']
    country = biz['location']['country']
    phone = biz['phone']
    price = biz['price']
    review_count = biz['review_count']
    rating = biz['rating']
    
    for cat in biz['categories']:
        categories.append({'alias': cat['alias'],'title': cat['title']})
        
        
    post = {
        'name': name,
        'address': address,
        'city': city,
        'state': state,
        'zip_code': zip_code,
        'country':country,
        'phone': phone,
        'price': price,
        'review_count': review_count,
        'rating':rating,
        'categories': categories
    }

    business.insert_one(post)
    categories.clear()